# FC Classification
1. Prepare Xy
2. fit a simple SVM classifier

In [11]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

## Prepare Xy

In [18]:
DATA = pd.read_csv('data/conn_df_correlation.csv', index_col='Unnamed: 0')

#Xy
X = DATA.drop(columns=['hypnosis_depth', 'procedure', 'description', 'session', 'condition', 'bids_id'])

hypnosis_depth_median = DATA['hypnosis_depth'].median()

y = DATA['hypnosis_depth'].apply(lambda x: 0 if x <= hypnosis_depth_median  else 1)

## SVM

In [19]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
from sklearn.feature_selection import VarianceThreshold

# pipeline
pipe = Pipeline([
    ('zerovar', VarianceThreshold(threshold=0.0)),
    ('scale', StandardScaler()),
    # ('select', SelectFromModel(SVC(kernel='linear'))),
    ('reduce', 'passthrough'),
    ('clf', SVC(kernel='linear', probability=True)),
    # ('clf', DummyClassifier(strategy='uniform'))
])

In [20]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe, X, y, cv=cv, scoring='accuracy', n_jobs=-1)

In [22]:
scores.mean()

0.63325